In [ ]:
ls ../scripts/five_bins/

In [ ]:
from rootpy.io import root_open
from rootpy.plotting import Hist, Canvas
from ROOT import TF1, TPaveText

dir_path = "../scripts/five_bins/"
tag_dict = { "CSVL": ["combinedSecondaryVertexBJetTags", 0.244 ],
             "CSVM": ["combinedSecondaryVertexBJetTags", 0.679 ],
             "CSVT": ["combinedSecondaryVertexBJetTags", 0.898 ],
             "JPL": ["jetProbabilityBJetTags", 0.275 ],
             "JPM": ["jetProbabilityBJetTags", 0.545 ],
             "JPT": ["jetProbabilityBJetTags", 0.79 ],
             "TCHPT" : ["trackCountingHighPurBJetTags", 3.41 ]}

In [ ]:

c = Canvas(1000,500)

s_name = "TCHPT"
tagger = tag_dict[s_name][0]
wp = tag_dict[s_name][1]


c_bins = [20, 30, 40, 50, 60, 70, 80,100, 120, 160, 210, 260, 320, 400, 500, 600, 800]
bins = [30, 50, 80, 120, 160, 320]

c_hist = Hist(c_bins)
c_hist.SetStats(False)
c_hist.SetAxisRange(0.7,1.3,"Y")
c_hist.GetXaxis().SetMoreLogLabels()
c_hist.GetXaxis().SetNoExponent()
c_hist.SetXTitle("Jet p_{T}[GeV/c]")
c_hist.SetYTitle("Data/Sim. b-tag SF")
c_hist.SetTitle(s_name + " - CMS prelim. 20 fb^{-1} at #sqrt{s} = 8 TeV ")
c_hist.Draw()

root_file = root_open("{}fit_result_{}_{}.root".format(dir_path, tagger, wp))
fit_result = root_file.nll
i_pars = fit_result.floatParsInit()
f_pars = fit_result.floatParsFinal()
hist = Hist( bins, title = "FtCM")
for i in range(2,1+len(bins)):
    mc_val = i_pars[f_pars.index("tag_eff_{}".format(i))].getVal()
    data_val = f_pars[f_pars.index("tag_eff_{}".format(i))].getVal()
    data_err = f_pars[f_pars.index("tag_eff_{}".format(i))].getError()
    sf = data_val/mc_val
    sf_err = data_err/mc_val
    hist[i-1] = (sf,sf_err)
hist.Draw("SAME E1")

ttbar_xs = f_pars[f_pars.index("xsec_TTbar_Summer13")].getVal()
ttbar_xs_err = f_pars[f_pars.index("xsec_TTbar_Summer13")].getError()
bkg_norm = f_pars[f_pars.index("kappa")].getVal()
bkg_norm_err = f_pars[f_pars.index("kappa")].getError()
text_box = TPaveText(0.60,0.7,0.87,0.87, "NDC")
text_box.AddText("pt-dependent FtCM")
text_box.AddText(" - #sigma_{t#bar{t}}" + 
                 " = {:4.0f} #pm {:2.0f}".format(ttbar_xs, ttbar_xs_err) +
                 " fb^{-1}")
text_box.AddText(" - #kappa" + 
                 " = {:5.2f} #pm {:5.2f}".format(bkg_norm,bkg_norm_err ))
text_box.Draw()

c.SetLogx(1)
c.GetPad(0).SetTickx()
c.GetPad(0).SetTicky()
t = c.GetPrimitive("title")
t.SetTextSize(0.04)
t.SetBBoxCenterX(300)
t.SetBBoxCenterY(30)
c